In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

from datetime import datetime
import openpyxl
import pandas as pd
import numpy as np
import time
import glob
import sys
import os

import sys
sys.path.append('../utils')
# from SharePoint import *
from hlpr import *

import plotly.express as px

In [2]:
files = glob.glob(os.path.join('../data', '*xlsx'))

# dfs_dict = {os.path.split(f)[-1]: pd.read_excel(f) for f in files}

**Files with Pivot Table as the First Sheet**

In [25]:
cols_list = []
for k in dfs_dict.keys():
    df = dfs_dict[k]
    cols = list(df.columns)
    cols_list.append()
    if cols[0] != 'Date':
        print(k,'\n')
        print(cols,'\n')
        print(f"{'-'*100}")

20210115190405944999_Amex Site Served Template_SBS_Group9_11.23.20.xlsx 

['Ad Name', '(Multiple Items)', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'] 

----------------------------------------------------------------------------------------------------
20210115191919501723_Amex Site Served Template_SBS_Group9_11.30.20.xlsx 

['Ad Name', '(All)', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'] 

----------------------------------------------------------------------------------------------------


**Column Name Frequencies**

In [79]:
df_cols = pd.DataFrame(cols_list).describe()
df_cols = pd.DataFrame(list(zip(df_cols.iloc[2, :], df_cols.iloc[3, :])), columns=['column', 'freq'])

fig = px.bar(data_frame=df_cols, x='column', y='freq', color='freq')
fig.show()

**Functions**

In [491]:
def parse_ss_filename(x):
    '''
    Parse Site Served template filename to timestamp and filename
    '''
    # find the last occurence of '.' and last occurence of '_'
    x_cut = x[:x.rfind('.')]
    if 'v' in x_cut[-2:].lower():
        filename_raw = x_cut[x_cut.find('_')+1:-3]
    else:
        filename_raw = x_cut[x_cut.find('_')+1:]

    # find the 1st '_'
    timestamp = float(x_cut[:x_cut.find('_')])

    return filename_raw, timestamp

In [492]:
def latest_ss_filename(filename_parse_list):
    df_filename = pd.DataFrame(filename_parse_list, columns=['filename', 'filename_raw', 'timestamp'])
    df_filename['max_timestamp'] = df_filename.groupby(['filename_raw'])['timestamp'].transform(max)
    df_filename['bool'] = df_filename['timestamp'] == df_filename['max_timestamp']
    filenames = list(df_filename[df_filename['bool']==True]['filename'])

    return filenames

In [505]:
def process_ss(ss_files):
    dfs_dict = {os.path.split(f)[-1]: pd.read_excel(f) for f in ss_files}

    filename_parse_list = []
    dfs_list = []
    for k in dfs_dict.keys():
        df = dfs_dict[k]
        df['filename'] = k
        filename_parse_results = parse_ss_filename(k)
        filename_parse_list.append([k, filename_parse_results[0], filename_parse_results[1]])
        cols = list(df.columns)
        if 'date' in [c.lower() for c in cols]:
            df.rename(columns={c: c.strip().lower() for c in df.columns}, inplace=True) # remove trailing space and make lower 
            dfs_list.append(df)

    # get list of the latest filenames
    latest_ss = latest_ss_filename(filename_parse_list)

    # combine and filter to the latest filename
    df = pd.concat(dfs_list, sort=True)
    df = df[df['filename'].isin(latest_ss)].reset_index(drop=True)

    # remove quartule and unnamed metrics
    col_filter = [c for c in df.columns if not('quartile' in c) and not('unnamed' in c)]
    df = df[col_filter]
    df['date'] = pd.to_datetime(df['date'])

    # split col by metrics, override, and dimensions
    metrics = ['clicks', 'spend', 'impressions', 'video plays', 'video completions']
    override = [c for c in df.columns if 'override' in c]
    dims = [c for c in df.columns if not(c in metrics) and not(c in override)]
    df = df[dims + metrics + override]

    # pair up the metric and override
    metric_override_pair = []
    for m in metrics:
        for o in override:
            if m in o:
                metric_override_pair.append([m, o])


    df_temp_list = []
    for p in metric_override_pair:
        df_temp = df.copy()
        df_temp = df_temp[dims + p]
        df_temp['metric'] = p[0]
        df_temp.rename(columns={p[0]:'value', p[1]:'override'}, inplace=True)
        df_temp_list.append(df_temp)

    df_trans = pd.concat(df_temp_list, sort=False)
    df_trans['value'] = df_trans['value'].fillna(0)
    df_trans['value'] = df_trans['value'].map(str).replace('-', None)
    df_trans['value'] = pd.to_numeric(df_trans['value'])

    # replace override and keep True
    df_trans['override'] = df_trans['override'].apply(lambda x: True if 'x' in str(x).lower() else False)
    df_trans = df_trans[df_trans['override']==True]
    df_trans.drop(columns=['filename', 'override'], inplace=True)
    df_trans.reset_index(drop=True, inplace=True)
    
    return df_trans

In [506]:
process_ss(files)

,campaign name,date,placement id,placement name,publisher (site),value,metric
0,USA_NAT_PRO_GC_GF_NON_GCS_2020_Q4_Programmatic...,2020-11-01,287698128,GCS_GCS_GCS 100 for 100_na_PLC_BRD_AWR_MAT_DV3...,Cadreon,0.0,clicks
1,USA_NAT_PRO_GC_GF_NON_GCS_2020_Q4_Programmatic...,2020-11-21,287698128,GCS_GCS_GCS 100 for 100_na_PLC_BRD_AWR_MAT_DV3...,Cadreon,1269.0,clicks
2,USA_NAT_PRO_GC_GF_NON_GCS_2020_Q4_Programmatic...,2020-11-22,287698128,GCS_GCS_GCS 100 for 100_na_PLC_BRD_AWR_MAT_DV3...,Cadreon,0.0,clicks
3,USA_NAT_PRO_GC_GF_NON_GCS_2020_Q4_Programmatic...,2020-11-23,287698128,GCS_GCS_GCS 100 for 100_na_PLC_BRD_AWR_MAT_DV3...,Cadreon,0.0,clicks
4,USA_NAT_PRO_GC_GF_NON_GCS_2020_Q4_Programmatic...,2020-11-24,287698128,GCS_GCS_GCS 100 for 100_na_PLC_BRD_AWR_MAT_DV3...,Cadreon,0.0,clicks
5,USA_NAT_PRO_GC_GF_NON_GCS_2020_Q4_Programmatic...,2020-11-25,287698128,GCS_GCS_GCS 100 for 100_na_PLC_BRD_AWR_MAT_DV3...,Cadreon,0.0,clicks
6,USA_NAT_PRO_GC_GF_NON_GCS_2020_Q4_Programmatic...,2020-11-26,287698128,GCS_GCS_GCS 100 for 100_na_PLC_BRD_AWR_MAT_DV3...,Cadreon,0.0,clicks
7,USA_NAT_PRO_GC_GF_NON_GCS_2020_Q4_Programmatic...,2020-11-27,287698128,GCS_GCS_GCS 100 for 100_na_PLC_BRD_AWR_MAT_DV3...,Cadreon,0.0,clicks
8,USA_NAT_PRO_GC_GF_NON_GCS_2020_Q4_Programmatic...,2020-11-28,287698128,GCS_GCS_GCS 100 for 100_na_PLC_BRD_AWR_MAT_DV3...,Cadreon,0.0,clicks
9,USA_NAT_PRO_GC_GF_NON_GCS_2020_Q4_Programmatic...,2020-11-29,287698128,GCS_GCS_GCS 100 for 100_na_PLC_BRD_AWR_MAT_DV3...,Cadreon,0.0,clicks


In [435]:
# df_filename.sort_values(by='filename_raw').to_excel('parsed_ss_filename.xlsx', index=False)